In [ ]:
"""
Convert AirSim's airsim_rec.txt to a clean CSV for
ego-UAV short-term trajectory prediction.

Directory layout expected:
[root]/
├─ raw_data/
│   ├─ airsim_rec.txt
│   └─ images/
│       └─ img_SimpleFlight__0_<timestamp>.png
│
└─ data_collector.py   <-- this file

Note: After running airsim API (which have import airsim), there is a high change ipynb/juypter notebook will not work, 
because there is a conflict between `airsim` and `ipykernel` module. So you need to reinstall it by: `pip install ipykernel`.
"""

from pathlib import Path
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R

# ----------------------- paths -----------------------
ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()
RAW_DIR    = ROOT / "raw_data"
TXT_FILE   = RAW_DIR / "airsim_rec.txt"
OUT_CSV    = RAW_DIR / "airsim_trajectory.csv"


In [2]:
# -------------------- load .txt ----------------------
df = pd.read_csv(
    TXT_FILE,
    sep=r"\t+",           # tab‑delimited
    engine="python",
    comment="#"
)

# ----------------- drop vehicle name ----------------
df = df.drop(columns=["VehicleName"], errors="ignore")

# ------------- quaternion ➜ roll‑pitch‑yaw ----------
def quat_to_rpy(qw, qx, qy, qz):
    # scipy expects (x, y, z, w)
    r = R.from_quat([qx, qy, qz, qw])
    return r.as_euler("xyz", degrees=False)   # roll, pitch, yaw

rpy = np.vstack([
    quat_to_rpy(qw, qx, qy, qz)
    for qw, qx, qy, qz in df[["Q_W","Q_X","Q_Y","Q_Z"]].values
])
df[["ROLL","PITCH","YAW"]] = rpy

# optional: drop quaternion columns
df = df.drop(columns=["Q_W","Q_X","Q_Y","Q_Z"])

# ---------- add sin/cos of yaw for continuity -------
df["YAW_SIN"] = np.sin(df["YAW"])
df["YAW_COS"] = np.cos(df["YAW"])

# ----------------- compute velocity -----------------
# AirSim TimeStamp is in micro‑seconds
df["TimeSec"] = df["TimeStamp"] * 1e-6

for axis in ["X","Y","Z"]:
    df[f"VEL_{axis}"] = df[f"POS_{axis}"].diff() / df["TimeSec"].diff()

# first row has NaNs after diff – drop or fill
df = df.dropna().reset_index(drop=True)

# --------------- save cleaned CSV -------------------
df.to_csv(OUT_CSV, index=False)
print(f"[✓]  Saved {len(df):,} rows to {OUT_CSV.relative_to(ROOT)}")


[✓]  Saved 2,171 rows to raw_data\airsim_trajectory.csv


In [3]:
# ------------------------------------------------------------------
# 6. Quick sanity check (optional)
# ------------------------------------------------------------------
print(df.head(5))


       TimeStamp     POS_X         POS_Y    POS_Z  \
0  1748915233389  0.000163  1.398570e-12 -4.95733   
1  1748915233440  0.001979  2.758130e-11 -4.94510   
2  1748915233491  0.006992  1.201170e-10 -4.93691   
3  1748915233542  0.016112  3.180450e-10 -4.93257   
4  1748915233593  0.029758  6.557500e-10 -4.93101   

                                     ImageFile          ROLL     PITCH  \
0  img_SimpleFlight__0_1748915233870793000.png  9.598816e-10 -0.069476   
1  img_SimpleFlight__0_1748915233920475700.png  2.332944e-09 -0.111565   
2  img_SimpleFlight__0_1748915233970870300.png  3.725878e-09 -0.141234   
3  img_SimpleFlight__0_1748915234022091400.png  5.126236e-09 -0.160287   
4  img_SimpleFlight__0_1748915234073948300.png  6.536680e-09 -0.170930   

            YAW       YAW_SIN  YAW_COS       TimeSec       VEL_X  \
0 -4.079368e-11 -4.079368e-11      1.0  1.748915e+06    5.319817   
1 -2.488914e-10 -2.488914e-10      1.0  1.748915e+06   35.612544   
2 -6.059280e-10 -6.059280e-10   